In [32]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain import PromptTemplate
import os

In [63]:
openaikey_env = open_ai_key
os.environ["OPENAI_API_KEY"] = open_ai_key
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", max_tokens = 3000)

In [68]:
background = \
'''
Who:The protagonist is [Alex], a young and ambitious astronaut. [Alex]grew up fascinated by the mysteries of space and had always dreamt of exploring the cosmos. Prior to becoming an astronaut, he worked as an engineer on Earth, where their passion for space exploration was nurtured through years of studying and experimentation.\n
Where:[Alex] and their loyal robotic companion, [Companion], find themselves aboard the ASV(Advanced Space Vessel) Starlight, a state-of-the-art spacecraft, drifting through the vast expanse of interstellar space. \n
The current time is set in the distant future, where humanity has begun its journey beyond the confines of the solar system, venturing into the uncharted territories of the cosmos.\n
Mission:The journey embarked upon by the protagonist was spurred by a monumental discovery made by a team of interstellar researchers. A mysterious signal originating from a distant star system, known only as the "Stellar Gateway," was detected. Eager to unlock the secrets of the universe, [Alex] volunteered for the mission to investigate the source of the signal.\n
What:As the ASV Starlight hurtled through the depths of space, its once pristine exterior was now marred by the scars of a violent cosmic storm. The storm's relentless fury had inflicted damage on multiple critical systems, plunging [Alex] and [Companion] into a perilous struggle for survival.\n
1. Windows shattered, exposing the void: The force of the storm shattered the ship's reinforced windows, sending fractured shards of transparent alloy scattering into the vacuum of space. The breach exposed the crew to the unforgiving cosmic expanse, with only emergency force fields preventing the total loss of atmospheric pressure within the vessel.\n
2. Water circulation system compromised: Within the ship's confines, the delicate balance of life-sustaining resources was thrown into disarray as the water circulation system suffered catastrophic damage. Leaking pipes and ruptured reservoirs caused precious water reserves to evaporate into the void, threatening to leave the crew without the essential resources for survival.\n
3. Power fluctuations plague the propulsion system: The storm's electromagnetic onslaught wreaked havoc on the ship's power distribution network, resulting in erratic fluctuations that crippled the propulsion system. Without a stable power supply, the engines sputtered and faltered, leaving the ASV Starlight adrift amidst the vast expanse of interstellar space.\n
In the wake of this devastation, [Alex] and [Companion] are faced with the daunting task of navigating the labyrinthine corridors of their damaged vessel, assessing the extent of the destruction, and devising a plan to effect repairs before their life support systems fail completely. With each passing moment, the cold void of space looms ever closer, underscoring the urgency of their mission and the dire consequences of failure.
'''

background = background.replace('[Alex]','Alex').replace('[Companion]', 'Mao')

In [71]:
frame_variable = ["topic", "background", "course"]
template_frame = \
'''
You are a {course} tutor, your audience is teenagers new to coding, your task here is to write 5-7 curriculum to teach topic {topic} in a interactive advanture scenario.\n
You need to create a whole story line following the background: {background}.\n
Each curriculum is designed to finish one mission of the advanture and teach one part of the topic.
You need to output a list of curriculums each with title and one sentence description of the mission and the feature you want to teach of that curriculum, the format should be: "1.[Title] - description:[Description]"
'''
prompt_frame = PromptTemplate(
    input_variables=frame_variable,
    template=template_frame,
)




content_variable = ["topic", "course", "tone", "title", "description", "time", "name"]
template_content = \
'''
You are a {course} tutor, your audience is teenagers new to coding, your task here is to write a series of curriculum to teach topic {topic} in a interactive advanture scenario.\n
You need to generate content for title {title} based on instruction {description}. The length of each curriculum should be around {time}. The missions here should include game parts that can be easily converted to puzzle games.\n
We have a virtual learning companion whose name is {name} to lead the mission, so you need to generate content in a EXTREMLY conversational way and in a {tone} tone, assign dialogue for the companion and leave blank for learner part.\
'''
prompt_content = PromptTemplate(
    input_variables=content_variable,
    template=template_content,
)



create_framework_chain = LLMChain(llm=llm,prompt=prompt_frame)


framework_result = create_framework_chain.run(course="Python",topic="Dictionary",background=background)

curr_list = framework_result.split('\n')
framework = {i.split('-')[0]:i.split('-')[1] for i in curr_list if i}

content_list = []
for title, description in framework.items():
    fill_content_chain = LLMChain(llm=llm,prompt=prompt_content)
    content_result = fill_content_chain.run(course="Python",topic="Dictionary",tone="encouraging",time="15 min", name="Mao",title=title,description=description)
    content_list.append((title, content_result))

In [72]:
with open('dictionary.md','w') as f:
    f.write('<h2>Background</h2>')
    f.write(background)
    f.write('<h2>syllabus</h2>')
    for k,v in framework.items():
        f.write(f'{k}<ul><li>{v}</li></ul>')
    f.write('<h2>curriculum</h2>')
    for title,content in content_list:
        f.write(f'<h3>{title}</h3>')
        f.write(content)